# Drone Trajectory Planner

In this project, we will develop the drone trajectory planner. This notebook serves as the main file for the project, where we will refer to the instructions and demonstrate our code.

Please follow week by week instructions, which includes writing the code in the `src/` folder.

In [1]:
# Import all the files and libraries required for the project
%load_ext autoreload
%autoreload 2
import copy
    
import numpy as np

from src.camera_utils import compute_image_footprint_on_surface, compute_ground_sampling_distance, project_world_point_to_image
from src.data_model import Camera, DatasetSpec
from src.plan_computation import compute_distance_between_images, compute_speed_during_photo_capture, generate_photo_plan_on_grid
from src.visualization import plot_photo_plan

import plotly.io as pio
pio.renderers.default = 'iframe'

# Week 1: Introduction

No code contribution expected this week

# Week 2: Implement data models for dataset specification and camera

For this week, we will model the dataset specification.

- Overlap: the ratio (in 0 to 1) of scene shared between two consecutive images.
- Sidelap: the ratio (in 0 to 1) of scene shared between two images in adjacent rows.
- Height: the height of the scan above the ground (in meters).
- Scan_dimension_x: the horizontal size of the rectangle to be scanned (in meters).
- Scan_dimension_y: the vertical size of the rectangle to be scanned (in meters).
- exposure_time_ms: the exposure time for each image (in milliseconds).


$\color{red}{\text{TODO: }}$ Implement `DatasetSpec` in `src/data_model.py`


In [2]:
# Model the nomimal dataset spec

overlap = 0.7
sidelap = 0.7
height = 30.48 # 100 ft
scan_dimension_x = 150
scan_dimension_y = 150
exposure_time_ms = 2 # 1/500 exposure time

dataset_spec = DatasetSpec(overlap, sidelap, height, scan_dimension_x, scan_dimension_y, exposure_time_ms)

print(f"Nominal specs: {dataset_spec}")

Nominal specs: DatasetSpec(overlap=0.7, sidelap=0.7, height=30.48, scan_dimension_x=150, scan_dimension_y=150, exposure_time_ms=2)


## Model the camera parameters

We want to model the following camera parameters in Python:
- focal length along x axis (in pixels)
- focal length along y axis (in pixels)
- optical center of the image along the x axis (in pixels)
- optical center of the image along the y axis (in pixels)
- Size of the sensor along the x axis (in mm)
- Size of the sensor along the y axis (in mm)
- Number of pixels in the image along the x axis
- Number of pixels in the image along the y axis

I recommend to use `dataclasses` ([Python documentation](https://docs.python.org/3/library/dataclasses.html), [Blog](https://www.dataquest.io/blog/how-to-use-python-data-classes/) to model these parameters.

$\color{red}{\text{TODO: }}$ Implement `Camera` in `src/data_model.py`

In [3]:
# Define the parameters for Skydio VT300L - Wide camera
# Ref: https://support.skydio.com/hc/en-us/articles/20866347470491-Skydio-X10-camera-and-metadata-overview
fx = 4938.56
fy = 4936.49
cx = 4095.5
cy = 3071.5
sensor_size_x_mm = 13.107 # single pixel size * number of pixels in X dimension
sensor_size_y_mm = 9.830 # single pixel size * number of pixels in Y dimension
image_size_x = 8192
image_size_y = 6144

camera_x10 = Camera(fx, fy, cx, cy, sensor_size_x_mm, sensor_size_y_mm, image_size_x, image_size_y)

In [4]:
print(f"X10 camera model: {camera_x10}")

X10 camera model: Camera(fx=4938.56, fy=4936.49, cx=4095.5, cy=3071.5, sensor_size_x_mm=13.107, sensor_size_y_mm=9.83, image_size_x_px=8192, image_size_y_px=6144)


# Week 3: Camera Operations

We plan to write utility functions to
- project a 3D world point to an image
- Compute image footprint on a surface
- Compute the Ground Sampling Distance

## Project 3D world points into the image


![Camera Projection](assets/image_projection.png)
Reference: [Robert Collins CSE483](https://www.cse.psu.edu/~rtc12/CSE486/lecture12.pdf)


Equations to implement:
$$ x = f_x \frac{X}{Z} $$
$$ y = f_y \frac{Y}{Z} $$
$$ u = x + c_x $$
$$ v = y + c_y $$

$\color{red}{\text{TODO: }}$ Implement function `project_world_point_to_image` in `src/camera_utils.py`

In [5]:
point_3d = np.array([25, -30, 50], dtype=np.float32)
expected_uv = np.array([6564.80, 109.60], dtype=np.float32)
uv = project_world_point_to_image(camera_x10, point_3d)

print(f"{point_3d} projected to {uv}")

assert np.allclose(uv, expected_uv, atol=1e-2)

[ 25. -30.  50.] projected to [6564.7803   109.60571]


## Compute Image Footprint on the surface

We have written code to *project* a 3D point into the image. The reverse operation is reprojection, where we take $(x, y)$ and compute the $(X, Y)$ for a given value of $Z$. Note that while going from 3D to 2D, the depth becomes ambiguous so we need the to specify the $Z$.

An image's footprint is the area on the surface which is captured by the image. We can take the two corners of the image and reproject them at a given distance to obtain the width and length of the image.

$\color{red}{\text{TODO: }}$ Implement function `compute_image_footprint_on_surface` in `src/camera_utils.py`

In [6]:
footprint_at_100m = compute_image_footprint_on_surface(camera_x10, 100)
expected_footprint_at_100m = np.array([165.88, 124.46], dtype=np.float32)

print(f"Footprint at 100m = {footprint_at_100m}")

assert np.allclose(footprint_at_100m, expected_footprint_at_100m, atol=1e-2)


Footprint at 100m = [165.87831 124.4609 ]


In [7]:
footprint_at_200m = compute_image_footprint_on_surface(camera_x10, 200)
expected_footprint_at_200m = expected_footprint_at_100m * 2

print(f"Footprint at 200m = {footprint_at_200m}")

assert np.allclose(footprint_at_200m, expected_footprint_at_200m, atol=1e-2)

Footprint at 200m = [331.75662 248.9218 ]


## Ground Sampling Distance

Ground sampling distance is the length of the ground (in m) captured by a single pixel. We have the image footpring (the dimensions of ground captured by the whole sensor, and the number of pixels along the horizontal and vertical dimension. Can we get GSD from these two quantities?

Note: Please return just one value of the GSD. Take the mininum of the values along the two axes.

In [8]:
gsd_at_100m = compute_ground_sampling_distance(camera_x10, 100)
expected_gsd_at_100m = 0.0202

print(f"GSD at 100m: {gsd_at_100m}")

assert np.allclose(gsd_at_100m, expected_gsd_at_100m, atol=1e-4)

GSD at 100m: 0.020248817279934883


## Bonus: Reprojection from 2D to 3D

If we have a 2d pixel location of a point along with the camera model, can we go back to 3D?
Do we need any additional information.


$\color{red}{\text{TODO: }}$ Implement function `reproject_image_point_to_world` in `src/camera_utils.py` and demonstrate it by running it in the notebook. Confirm that your reprojection + projection function are by projecting a 3d point to image, and reprojecting it back to 3D.

# Week 4: Compute Distance Between Photos

The overlap and sidelap are the ratio of the dimensions shared between two photos. We already know the footprint of a single image at a given distance. Can we convert the ratio into actual distances? And how does the distance on the surface relate to distance travelled by the camera?

$\color{red}{\text{TODO: }}$ Implement `compute_distance_between_images` in `src/plan_computation.py`



In [9]:
computed_distances = compute_distance_between_images(camera_x10, dataset_spec)
expected_distances = np.array([15.17, 11.38], dtype=np.float32)

print(f"Computed distance for X10 camera with nominal dataset specs: {computed_distances}")

assert np.allclose(computed_distances, expected_distances, atol=1e-2)

Computed distance for X10 camera with nominal dataset specs: [15.167914 11.380706]


$\color{red}{\text{TODO: }}$ define >=2 more specifications/camera parameters and check the computed distances. Does that align with your expections


In [10]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

computed_distances_ = compute_distance_between_images(camera_, dataset_spec_)
print(f"Computed distance: {computed_distances_}")

Computed distance: [15.167914 11.380706]


In [11]:
# Experiment A: increase focal length by 50%
camera_A = copy.copy(camera_x10)

camera_A.fx = camera_A.fx * 1.5
camera_A.fy = camera_A.fy * 1.5

dataset_A = copy.copy(dataset_spec)

computed_distances_A = compute_distance_between_images(camera_A, dataset_A)

print("Experiment A — focal length *1.5")
print("New fx, fy:", camera_A.fx, camera_A.fy)
print("Computed distances (x, y):", computed_distances_A)

Experiment A — focal length *1.5
New fx, fy: 7407.84 7404.735
Computed distances (x, y): [10.111942   7.5871367]


Experiment A — increased focal length by 50%:
Increasing the camera’s focal length narrows the field of view, causing the footprint on the ground to shrink.
As expected, the computed distances between image centers decreased, meaning flight lines would need to be closer to maintain the same overlap.
This results in more images required to cover the same area, but each image has higher ground detail (better GSD).

In [12]:
# Experiment B: double flight height
camera_B = copy.copy(camera_x10)
dataset_B = copy.copy(dataset_spec)
dataset_B.height = dataset_B.height * 2.0

computed_distances_B = compute_distance_between_images(camera_B, dataset_B)

print("Experiment B — height *2.0")
print("New height:", dataset_B.height)
print("Computed distances (x, y):", computed_distances_B)

Experiment B — height *2.0
New height: 60.96
Computed distances (x, y): [30.335829 22.761412]


Experiment B — doubled flight height:
Doubling the flight height increases the ground footprint proportionally, so the distances between image centers roughly doubled.
This allows for fewer flight lines to cover the same area, speeding up surveys, but causes reduced ground resolution and potentially greater atmospheric or geometric distortions.

## Bonus: Non-Nadir photos

We have solved for the distance assuming that the camera is facing straight down to the ground. This is called [Nadir scanning](https://support.esri.com/en-us/gis-dictionary/nadir). However, in practise we might want a custom gimbal angle.

Your bonus task is to make the distance computation general. Introduce a double `camera_angle` parameter (which is the angle from the X-axis) in the dataset specification, and work out how to adapt your computation. Feel free to reach out to Ayush to discuss ideas and assumptions!

![Non Nadir Footprint](assets/non_nadir_gimbal_angle.png)

# Week 5: Compute Maximum Speed For Blur Free Photos

To restrict motion blur due to camera movement to tolerable limits, we need to restrict the speed such that the image contents move less than 1px away. 

How much does 1px of movement translate to movement of the scene on the ground? It is the ground sampling distance!
From previous week, we know that this is the maximum movement the camera can have. 
We have the distance now. To get speed we need to divide it with time. Do we have time already in our data models?

$\color{red}{\text{TODO: }}$ Implement `compute_speed_during_photo_capture` in `src/plan_computation.py`.

In [13]:
computed_speed = compute_speed_during_photo_capture(camera_x10, dataset_spec, allowed_movement_px=1)
expected_speed = 3.09

print(f"Computed speed during photo captures: {computed_speed:.2f}")

assert np.allclose(computed_speed, expected_speed, atol=1e-2)

Computed speed during photo captures: 3.09


$\color{red}{\text{TODO: }}$ define >= 2more specifications/camera parameters and check the computed distances. Does that align with your expectations


In [14]:
camera_ = copy.copy(camera_x10)
dataset_spec_ = copy.copy(dataset_spec)

computed_speed_ = compute_speed_during_photo_capture(camera_, dataset_spec_)
print(f"Computed distance: {computed_speed_:.2f}")

Computed distance: 3.09


In [15]:
# Experiment A: increase focal length by 50%
camera_A = copy.copy(camera_x10)
camera_A.fx = camera_A.fx * 1.5
camera_A.fy = camera_A.fy * 1.5

dataset_A = copy.copy(dataset_spec)

computed_speed_A = compute_speed_during_photo_capture(camera_A, dataset_A)
print("Experiment A — focal length *1.5")
print("New fx, fy:", camera_A.fx, camera_A.fy)
print(f"Computed speed: {computed_speed_A:.2f} m/s\n")

Experiment A — focal length *1.5
New fx, fy: 7407.84 7404.735
Computed speed: 2.06 m/s



Experiment A — Increasing the focal length reduces the field of view, leading to a smaller ground footprint and finer GSD. Because each pixel now covers a smaller area on the ground, the allowable motion for 1px blur decreases. As expected, the computed blur-free speed decreases, meaning the drone must move slower to maintain sharp images.

In [16]:
# Experiment B: double flight height
camera_B = copy.copy(camera_x10)
dataset_B = copy.copy(dataset_spec)
dataset_B.height = dataset_B.height * 2.0

computed_speed_B = compute_speed_during_photo_capture(camera_B, dataset_B)
print("Experiment B — height *2.0")
print("New height:", dataset_B.height)
print(f"Computed speed: {computed_speed_B:.2f} m/s\n")

Experiment B — height *2.0
New height: 60.96
Computed speed: 6.17 m/s



Experiment B — doubled flight height: Increasing the flight height increases the ground footprint and GSD, meaning each pixel covers more area. This allows for greater camera movement (in meters) before exceeding the 1px blur threshold. As expected, the computed allowable speed roughly doubles — the drone can move faster while still keeping images sharp.

# Week 6: Generate Full Flight Plans  

We now have all the tools to generate the full flight plan.

Steps for this week:
1. Define the `Waypoint` data model. What attributes should the data model have?
   1. For Nadir scans, just the position of the camera is enough as we will always look drown to the ground.
   2. For general case (bonus), we also need to define where the drone will look at.
3. Implement the function `generate_photo_plan_on_grid` to generate the full plan.
   1. Compute the maximum distance between two images, horizontally and vertically.
   2. Layer the images such that we cover the whole scan area. Note that you need to take care when the scan dimension is not a multiple of distance between images. Example: to cover 45m length with 10m between images, we would need 4.5 images. Not possible. 4 images would not satisfy the overlap, so we should go with 5. How should we arrange 5 images in the given 45m.
   3. Assign the speed to each waypoint.

$\color{red}{\text{TODO: }}$ Implement:
- `Waypoint` in `src/data_model.py`
- `generate_photo_plan_on_grid` in `src/plan_computation.py`.

In [17]:
computed_plan = generate_photo_plan_on_grid(camera_x10, dataset_spec) 

print(f"Computed plan with {len(computed_plan)} waypoints")

Computed plan with 140 waypoints


In [18]:
MAX_NUM_WAYPOINTS_TO_PRINT = 20

for idx, waypoint in enumerate(computed_plan[:MAX_NUM_WAYPOINTS_TO_PRINT]):
    print(f"Idx {idx}: {waypoint}")
if len(computed_plan) >= MAX_NUM_WAYPOINTS_TO_PRINT:
    print("...")

Idx 0: Waypoint(position=(np.float32(0.0), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 1: Waypoint(position=(np.float32(15.167914), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 2: Waypoint(position=(np.float32(30.335829), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 3: Waypoint(position=(np.float32(45.503742), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 4: Waypoint(position=(np.float32(60.671658), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 5: Waypoint(position=(np.float32(75.83957), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 6: Waypoint(position=(np.float32(91.007484), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 7: Waypoint(position=(np.float32(106.1754), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 8: Waypoint(position=(np.float32(121.343315), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 9: Waypoint(position=(np.float32(136.51123), np.float32(0.0), 30.48), speed=3.085919888690114)
Idx 10: Waypoint(

## Bonus: Time computation 

if you have some time, you can implement a time computation function. We can make the drone fly as fast as possible between photos, but make sure it can decelerate back to the required speed at the photos. Please use the following data: 
- Max drone speed: 16m/s.
- Max acceleration: 3.5 m/s^2.

Hint: you might need to use a trapezoidal/triangular speed profile

# Week 7: Visualize Flight Plans

This week, we will use a third party plotting framework called [Plotly](https://plotly.com/python/) to visualize our plans. Please follow this [tutorial](https://www.kaggle.com/code/kanncaa1/plotly-tutorial-for-beginners) to gain some basic experience with Plotly, and then come up with your own visualization function. You are free to choose to come up with your own visualization, and use something other than Plotly.

$\color{red}{\text{TODO: }}$ Implement `plot_photo_plan` in `src/visualization.py`

In [19]:
fig = plot_photo_plan(computed_plan)
fig.show()

$\color{red}{\text{TODO: }}$ Perform the following experiments (and any other you can think of) where we change just one parameter of the input camera/dataset specification and observe the change in the output plan. 

1. Change overlap and confirm it affects the consecutive images
2. Change sidelap and confirm it does not affect the consecutive images
3. Change the height of the scan and document the affect on scan plans
4. Change exposure time

Each experiment should specify: 
1. Input params you are changing
2. Impact you observe
3. explanation behind the change in output (intuition based or a text explanation is preffered over using equations)
4. Practical implication of the correlation: how can I drone pilot use this result

In [20]:
# Experiment 1: increase overlap
import copy

camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.overlap = 0.9

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Experiment 1 - increased overlap

Input parameter changed: overlap (0.7 → 0.9)

Impact observed: Waypoints are closer together horizontally (drone takes more photos per row).

Explanation: Higher overlap means each photo must cover more of the previous one, so spacing decreases.

Practical implication: Better map quality but more battery and storage used.

In [21]:
# Experiment 2: change sidelap
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.sidelap = 0.8

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Experiment 2 – change sidelap

Input parameter changed: sidelap (0.6 → 0.8)

Impact observed: More rows are added to cover same area; left-right spacing unchanged.

Explanation: Sidelap only affects spacing between flight rows, not along a row.

Practical implication: Better vertical alignment for 3D reconstruction, but more flight time.

In [22]:
# Experiment 3: change scan height
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.height = 25

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Experiment 3 - change scan height

Input parameter changed: height (50m → 25m)

Impact observed: More waypoints needed to cover the same area.

Explanation: Lower altitude translates to smaller ground coverage per image, meaning more passes are needed.

Practical implication: Higher detail datasets but more battery used and longer mission time.

In [23]:
# Experiment 4: change exposure time
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

dataset_spec_.exposure_time_ms = 1000

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()

Experiment 4 – change exposure time

Input parameter changed: exposure time (10ms → 1000ms)

Impact observed: Drone slows down significantly.

Explanation: Longer exposure means there is a risk of motion blur unless speed is reduced.

Practical implication: In low light, drone must fly slower.

In [24]:
# Experiment 5: change focal length
camera_ = copy.deepcopy(camera_x10)
dataset_spec_ = copy.deepcopy(dataset_spec)

camera_.fx = camera_.fx * 1.5
camera_.fy = camera_.fy * 1.5

fig = plot_photo_plan(generate_photo_plan_on_grid(camera_, dataset_spec_))
fig.show()


Experiment 5 - change focal length
Input parameter changed: camera.fx and camera.fy multiplied by 1.5 (50% longer focal length)

Impact observed: waypoints become more dense, more images required.

Explanation: Longer focal length narrows the camera’s field of view.

Practical implication: Higher detail, but more flight time.